TODO : 
- Pré processer mes text pour ne plus avoir les trucs pourris avant d'utiliser le tokenizer de keras
- Utiliser des embeddings pré réalisés --> Notamment fast text et initialiser ma couche d'embeddings à partir de fast text
    https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html
    https://www.kaggle.com/sbongo/do-pretrained-embeddings-give-you-the-extra-edge
    https://www.kaggle.com/mschumacher/using-fasttext-models-for-robust-embeddings
    https://www.kaggle.com/sterby/fasttext-like-baseline-with-keras-lb-0-053
    
- Utiliser un lstm bidirectionnel / GRU
    https://www.kaggle.com/CVxTz/keras-bidirectional-lstm-baseline-lb-0-069
    https://machinelearningmastery.com/develop-bidirectional-lstm-sequence-classification-python-keras/
- Augmenter la profondeur de mon NN
- Rajouter des features (taille du texte, nombre de caractères pourris (@-+.) 
- Essayer avec un CNN 
- Utiliser des méthodes de bagging et voir ce que ca fait

In [3]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [4]:
import numpy as np
import pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, GRU, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import EarlyStopping

In [5]:
# Load Data
train_data_path = "data/train.csv"
test_data_path = "data/test.csv"

train_set = pd.read_csv(train_data_path)

In [6]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
Y_train = train_set[train_set.columns[2:]]
list_sentences_train = train_set["comment_text"]

In [7]:
max_features = 50000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))

list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)

In [8]:
maxlen = 300
X_train = pad_sequences(list_tokenized_train, maxlen=maxlen)

In [9]:
class DataGenerator:
    
    def __init__(self):
        self.index_train = 0
    
    def generate_data(self, batch_size, train):
        while True:
            comments = []
            labels = []
            if self.index_train + batch_size < len(X_train): 
                comments.append(X_train[self.index_train:self.index_train + batch_size])
                labels.append(Y_train[self.index_train:self.index_train + batch_size])
                self.index_train += batch_size
            else:
                rest_train = (self.index_train + batch_size) % len(X_train)
                comments = [np.append(X_train[self.index_train:], X_train[:rest_train], axis=0)]
                labels = [np.append(Y_train[self.index_train:], Y_train[:rest_train], axis=0)]
                self.index_train = rest_train
            yield comments, labels

In [10]:
embed_size = 128
input_layer = Input(shape=(maxlen,))
embed_layer = Embedding(max_features, embed_size)(input_layer)
x = GRU(64, return_sequences=True, name='gru_layer')(embed_layer)
x = GlobalMaxPool1D()(x)
x = Dropout(0.2)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(len(list_classes), activation="sigmoid")(x)

In [11]:
# define early stopping callback
earlystop = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=5)
callbacks_list = [earlystop]

In [12]:
model = Model(inputs=input_layer, outputs=x)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 300, 128)          6400000   
_________________________________________________________________
gru_layer (GRU)              (None, 300, 64)           37056     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [14]:
data_generator = DataGenerator()

In [16]:
batch_size = 64
epochs = 20
model_info = model.fit_generator(data_generator.generate_data(batch_size,True),
                        samples_per_epoch=200, nb_epoch=epochs)

/home/guillaume/.local/lib/python3.5/site-packages/ipykernel_launcher.py:4: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  after removing the cwd from sys.path.
/home/guillaume/.local/lib/python3.5/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=20, steps_per_epoch=200)`
  after removing the cwd from sys.path.


Epoch 1/20
200/200 [==============================] - 54s 271ms/step - loss: 0.0425 - acc: 0.9842
Epoch 2/20
200/200 [==============================] - 49s 244ms/step - loss: 0.0421 - acc: 0.9840
Epoch 3/20
200/200 [==============================] - 56s 278ms/step - loss: 0.0437 - acc: 0.9836
Epoch 4/20
200/200 [==============================] - 50s 252ms/step - loss: 0.0393 - acc: 0.9848
Epoch 5/20
200/200 [==============================] - 51s 254ms/step - loss: 0.0433 - acc: 0.9835
Epoch 6/20
200/200 [==============================] - 52s 258ms/step - loss: 0.0400 - acc: 0.9842
Epoch 7/20
200/200 [==============================] - 53s 265ms/step - loss: 0.0407 - acc: 0.9842
Epoch 8/20
200/200 [==============================] - 57s 284ms/step - loss: 0.0370 - acc: 0.9850
Epoch 9/20
200/200 [==============================] - 58s 290ms/step - loss: 0.0360 - acc: 0.9864
Epoch 10/20
200/200 [==============================] - 54s 269ms/step - loss: 0.0377 - acc: 0.9855
Epoch 11/20
200/200

In [17]:
# Save model
model.save('gru_toxic_comments_classifier.h5')

In [24]:
model.evaluate(X_train, Y_train)

159571/159571 [==============================] - 798s 5ms/step


[0.04652112938131759, 0.98278716955049805]

In [18]:
test_set = pd.read_csv(test_data_path)
list_sentences_test = test_set["comment_text"]

list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

maxlen = 300
X_test = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [19]:
Y_test = model.predict(X_test)

In [20]:
Y_test.shape

(153164, 6)

In [21]:
final_df = pd.DataFrame()

In [22]:
final_df['toxic'] = Y_test[:, 0]
final_df['severe_toxic'] = Y_test[:, 1]
final_df['obscene'] = Y_test[:, 2]
final_df['threat'] = Y_test[:, 3]
final_df['insult'] = Y_test[:, 4]
final_df['identity_hate'] = Y_test[:, 5]

In [23]:
final_df.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0.999713,5.007983e-01,0.993811,1.249188e-01,9.719996e-01,2.779484e-01
1,0.000009,3.886579e-10,0.000006,2.055729e-08,8.452794e-07,1.429677e-08
2,0.000063,4.374673e-08,0.000043,2.233486e-06,8.512913e-06,3.693515e-07
3,0.000253,1.019396e-08,0.000027,8.840391e-07,1.756022e-05,5.647765e-07
4,0.000111,6.166028e-09,0.000023,4.898149e-07,5.668001e-06,2.268521e-07


In [24]:
submissions = pd.read_csv("data/sample_submission.csv")
final_df.insert(0, 'id', submissions['id'])

In [25]:
final_df.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.999713,5.007983e-01,0.993811,1.249188e-01,9.719996e-01,2.779484e-01
1,0000247867823ef7,0.000009,3.886579e-10,0.000006,2.055729e-08,8.452794e-07,1.429677e-08
2,00013b17ad220c46,0.000063,4.374673e-08,0.000043,2.233486e-06,8.512913e-06,3.693515e-07
3,00017563c3f7919a,0.000253,1.019396e-08,0.000027,8.840391e-07,1.756022e-05,5.647765e-07
4,00017695ad8997eb,0.000111,6.166028e-09,0.000023,4.898149e-07,5.668001e-06,2.268521e-07


In [26]:
final_df.to_csv('lstm_df.csv', index = False)